In [1]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install langid
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!sudo apt-get install hunspell
!sudo apt-get install hunspell-th

Reading package lists... Done
Building dependency tree       
Reading state information... Done
hunspell is already the newest version (1.7.0-2build2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
hunspell-th is already the newest version (1:6.4.3-1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
from tqdm import tqdm 
import pickle
from datasets import load_metric
import subprocess
import sys
import langid
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [6]:
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k_2.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k_2.pkl', 'rb'))

In [7]:
ds_mlm = ds_mlm[10576:]
ds_tag = ds_tag[10576:]
NUM_SAMPLE = ds_mlm.shape[0]
NUM_SAMPLE

5000

In [8]:
msp_type = pickle.load(open('drive/MyDrive/AIBuilders/json/msp_type.pkl', 'rb'))
msp_type = msp_type[10576:]
msp_word = pickle.load(open('drive/MyDrive/AIBuilders/json/msp_word.pkl', 'rb'))
msp_word = msp_word[10576:]
print(len(msp_type), len(msp_word))

5000 5000


In [9]:
def spell(word,lang):
    if sys.platform == 'win32':
        cmd = "echo "+word+" | hunspell -d "+lang
    else:
        cmd = 'echo "'+word+'" | hunspell -d '+lang
    getoutput = subprocess.getoutput(cmd)
    del cmd
    get = getoutput.split("\n")
    del get[0]
    if get[0] == '*':
        getoutput = "No Suggestions"
    else:
        if get[1] == "":
            del get[1]
        get = get[0].split(":")
        del get[0]
        getoutput = get[0].replace(" ","")
        getoutput = getoutput.split(",")
    del get
    return getoutput
Input = spell("appld","")
print(Input)
InputTH = spell("สถาปัตยกรรม","th_TH")
print(InputTH)
trueth = spell("กิร","th_TH")
print(trueth)

['apple', 'apply', 'applaud']
No Suggestions
['ริก', 'กร', 'กิระ']


In [ ]:
msp_type_dict = {}
msp_type_dict_full = {}
msp_type_dict_fill = {}
sent_id_type = {}
msp_word_dict = {}
msp_word_dict_full = {}
msp_word_dict_fill = {}

len_msp_type = len(msp_type)
for i in range(len_msp_type):
  len_a = len(msp_type[i])
  for j in range(len_a):
    k = msp_type[i][j][0]
    if not k in msp_type_dict:
      msp_type_dict[k] = 1
      msp_type_dict_full[k] = 0
      msp_type_dict_fill[k] = 0
      sent_id_type[k] = []
    else:
      msp_type_dict[k] += 1
    
    len_b = len(msp_word[i][j])
    for k in range(len_b):
      x = msp_word[i][j][k]
      if not x in msp_word_dict:
        msp_word_dict[x] = 1
        msp_word_dict_full[x] = 0
        msp_word_dict_fill[x] = 0
      else:
        msp_word_dict[x] += 1

msp_word_dict_wrong = msp_word_dict_fill.copy()
print(msp_type_dict)
print(msp_type_dict_fill)
print(msp_type_dict_full)
msp_word_dict = dict(sorted(msp_word_dict.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict)
print(msp_word_dict_fill)
print(msp_word_dict_full)


bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

# NUM_SAMPLE=10
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    if langid.classify(text_token[j])[0] == 'th':
      try:
        corrected = spell(text_token[j],"th_TH")
      except:
        print(f"err hunspell: {text_token[j]}")
      else:
        if not corrected == "No Suggestions":
          # print(f"{text_token[j]} => {corrected[0]}")
          text_token[j] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for j in range(labels_len):
    if not text_token[j] == original_token[j]: #change
      denom += 1
      if text_token[j] == labels_token[j]:
        numer += 1
        TP += 1
      elif not text_token[j] == labels_token[j]:
        FP += 1
    elif text_token[j] == original_token[j]: #no change
      if text_token[j] == labels_token[j]:
        TN += 1
      elif not text_token[j] == labels_token[j]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred


  # print(f"MSP TYPE: {msp_type[i]}")
  original_id = text_id.copy()
  labels = labels_id.copy()
  predicted_id = tokenizer.convert_tokens_to_ids(text_token)
  # print(f"ORIG ({len(original_id)}): {original_id}")
  # print(f"PRED ({len(predicted_id)}): {predicted_id}")
  # print(f"REFE ({len(labels)}): {labels}")
  for w in msp_type[i]:
    msp_type_dict_full[w[0]] += 1
    flag = 0
    a = []
    for j in range(w[1]+1, w[2]+1):
      a.append((original_id[j], predicted_id[j]))
      # print(tokenizer.convert_ids_to_tokens(predicted_id[j]), tokenizer.convert_ids_to_tokens(labels[j]))
      if predicted_id[j] != labels[j]:
        flag = 1
        break
    if flag == 0:
      msp_type_dict_fill[w[0]] += 1
      if i not in sent_id_type[w[0]]: #it may look too little because it only takes full marks
        sent_id_type[w[0]].append([i, a])


  for j in range(labels_len):
    if labels[j] != original_id[j]: #change
      word_original = tokenizer.convert_ids_to_tokens(original_id[j])
      word_original = word_original.replace("▁", "")
      word_predicted = tokenizer.convert_ids_to_tokens(predicted_id[j])
      word_predicted = word_predicted.replace("▁", "")
      word_labels = tokenizer.convert_ids_to_tokens(labels[j])
      word_labels = word_labels.replace("▁", "")
      if (word_original, word_labels) in msp_word_dict_fill: #have key
        msp_word_dict_full[(word_original, word_labels)] += 1
        if labels[j] == predicted_id[j]: #correct
          msp_word_dict_fill[(word_original, word_labels)] += 1
        elif labels[j] != predicted_id[j]: #incorrect
          msp_word_dict_wrong[(word_original, word_labels)] += 1


  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")
print(msp_type_dict_full)
print(msp_type_dict_fill)
msp_word_dict_full = dict(sorted(msp_word_dict_full.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_fill = dict(sorted(msp_word_dict_fill.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_wrong = dict(sorted(msp_word_dict_wrong.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict_full)
print(msp_word_dict_fill)
print(msp_word_dict_wrong)
print(sent_id_type)

{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 487, ('คับ', 'ครับ'): 466, ('คะ', 'ค่ะ'): 345, ('เบอ', 'เบอร์'): 315, ('มั้ย', 'ไหม'): 205, ('สมัค', 'สมัคร'): 186, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 136, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('โทรศัพ', 'โทรศัพท์'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('เคดิต', 'เครดิต'): 62, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'): 55, ('อยุ่', 'อยู่'): 53, ('บันชี', 'บัญชี'): 53, ('ไหม่', 'ใหม่'): 51, ('อ้ะ', 'อะ'): 49, ('รุ้', 'รู้'): 48, ('ก้อ', 'ก็'): 48, ('แร้ว', 'แล้ว'): 46, ('ใหน', 'ไหน

  0%|          | 3/5000 [00:01<30:16,  2.75it/s]

err hunspell: ครับ


  0%|          | 4/5000 [00:01<29:44,  2.80it/s]

err hunspell: ้


...

  1%|          | 57/5000 [00:29<51:55,  1.59it/s]

err hunspell: อีก


  1%|          | 62/5000 [00:31<36:37,  2.25it/s]

err hunspell: ธนาคาร


  1%|▏         | 66/5000 [00:33<31:43,  2.59it/s]

In [10]:
def ids_to_tokens(tokenized_text):
  a = tokenizer.convert_ids_to_tokens(tokenized_text)
  # a.remove("<s>")
  # a.remove("</s>")
  # if a[0] == '▁':
  #   a.pop(0)
  return a

In [11]:
for key in sent_id_type:
  print(f"=========[{key}]==========")
  for sent_id, words in sent_id_type[key]:
    text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
    labels = ds_mlm.iloc[sent_id]['labels']
    text = [k for k in text if k != 1]
    labels = [k for k in labels if k != 1]
    original = [ids_to_tokens(text)]
    references = [[ids_to_tokens(labels)]]
    for i in words:
      print(f"MSP: {tokenizer.convert_ids_to_tokens(i[0])} => {tokenizer.convert_ids_to_tokens(i[1])}")
    print(original)
    print(references)
    print("-------------------")

NameError: ignored

In [ ]:
pickle_hunspell_json = {}
pickle_hunspell_json['msp_type_dict_full'] = msp_type_dict_full
pickle_hunspell_json['msp_type_dict_fill'] = msp_type_dict_fill
pickle_hunspell_json['msp_word_dict_full'] = msp_word_dict_full
pickle_hunspell_json['msp_word_dict_fill'] = msp_word_dict_fill
pickle_hunspell_json['msp_word_dict_wrong'] = msp_word_dict_wrong
pickle_hunspell_json['sent_id_type'] = sent_id_type

pickle.dump(pickle_hunspell_json, open('pickle_hunspell_json.pkl', 'wb'))

In [12]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

FILE = "drive/MyDrive/AIBuilders/json/tagging_json_400.pth"
tagging_model = BertModel()
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.9.output.LayerNorm.bias', 'roberta.encoder.layer.8.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.self.query.bias', 'roberta.encoder.layer.5.intermediate.dense.bias', 'roberta.encoder.layer.7.attention.self.key.weight', 'roberta.encoder.layer.9.intermediate.dense.bias', 'roberta.encoder.layer.7.attention.output.LayerNorm.bias', 'roberta.encoder.layer.9.intermediate.dense.weight', 'roberta.encoder.layer.3.intermediate.dense.weight', 'roberta.encoder.layer.8.intermediate.dense.bias', 'roberta.encoder.layer.10.intermediate.dense.weight', 'roberta.encoder.layer.4.attention.self.value.weight', 'roberta.embeddings.LayerNorm.bias',

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(25354, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [13]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
msp_type_dict = {}
msp_type_dict_full = {}
msp_type_dict_fill = {}
sent_id_type = {}
msp_word_dict = {}
msp_word_dict_full = {}
msp_word_dict_fill = {}

len_msp_type = len(msp_type)
for i in range(len_msp_type):
  len_a = len(msp_type[i])
  for j in range(len_a):
    k = msp_type[i][j][0]
    if not k in msp_type_dict:
      msp_type_dict[k] = 1
      msp_type_dict_full[k] = 0
      msp_type_dict_fill[k] = 0
      sent_id_type[k] = []
    else:
      msp_type_dict[k] += 1
    
    len_b = len(msp_word[i][j])
    for k in range(len_b):
      x = msp_word[i][j][k]
      if not x in msp_word_dict:
        msp_word_dict[x] = 1
        msp_word_dict_full[x] = 0
        msp_word_dict_fill[x] = 0
      else:
        msp_word_dict[x] += 1

msp_word_dict_wrong = msp_word_dict_fill.copy()
print(msp_type_dict)
print(msp_type_dict_fill)
print(msp_type_dict_full)
msp_word_dict = dict(sorted(msp_word_dict.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict)
print(msp_word_dict_fill)
print(msp_word_dict_full)

bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

# NUM_SAMPLE=10
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  mask = ds_mlm.iloc[i]['attention_mask']
  mask = [k for k in mask if k != 0]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]

  i_f = evaluate_one_text(tagging_model, text_id, mask, labels_id)
  # print(i_f)
  i_f_len = len(i_f)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(i_f_len):
    text_token[j+1] = text_token[j+1].replace("▁", " ")
    if(i_f[j] == 'i'):    
      if langid.classify(text_token[j+1])[0] == 'th':
        try:
          corrected = spell(text_token[j+1],"th_TH")
        except:
          print(f"err hunspell: {text_token[j+1]}")
        else:
          if not corrected == "No Suggestions":
            # print(f"{text_token[j+1]} => {corrected[0]}")
            text_token[j+1] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for j in range(labels_len):
    if not text_token[j] == original_token[j]: #change
      denom += 1
      if text_token[j] == labels_token[j]:
        numer += 1
        TP += 1
      elif not text_token[j] == labels_token[j]:
        FP += 1
    elif text_token[j] == original_token[j]: #no change
      if text_token[j] == labels_token[j]:
        TN += 1
      elif not text_token[j] == labels_token[j]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  # print(references)
  # print(original)
  # print(predictions)

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred


  # print(f"MSP TYPE: {msp_type[i]}")
  original_id = text_id.copy()
  labels = labels_id.copy()
  predicted_id = tokenizer.convert_tokens_to_ids(text_token)
  # print(f"ORIG ({len(original_id)}): {original_id}")
  # print(f"PRED ({len(predicted_id)}): {predicted_id}")
  # print(f"REFE ({len(labels)}): {labels}")
  for w in msp_type[i]:
    msp_type_dict_full[w[0]] += 1
    flag = 0
    a = []
    for j in range(w[1]+1, w[2]+1):
      a.append((original_id[j], predicted_id[j]))
      # print(tokenizer.convert_ids_to_tokens(predicted_id[j]), tokenizer.convert_ids_to_tokens(labels[j]))
      if predicted_id[j] != labels[j]:
        flag = 1
        break
    if flag == 0:
      msp_type_dict_fill[w[0]] += 1
      if i not in sent_id_type[w[0]]: #it may look too little because it only takes full marks
        sent_id_type[w[0]].append([i, a])


  for j in range(labels_len):
    if labels[j] != original_id[j]: #change
      word_original = tokenizer.convert_ids_to_tokens(original_id[j])
      word_original = word_original.replace("▁", "")
      word_predicted = tokenizer.convert_ids_to_tokens(predicted_id[j])
      word_predicted = word_predicted.replace("▁", "")
      word_labels = tokenizer.convert_ids_to_tokens(labels[j])
      word_labels = word_labels.replace("▁", "")
      if (word_original, word_labels) in msp_word_dict_fill: #have key
        msp_word_dict_full[(word_original, word_labels)] += 1
        if labels[j] == predicted_id[j]: #correct
          msp_word_dict_fill[(word_original, word_labels)] += 1
        elif labels[j] != predicted_id[j]: #incorrect
          msp_word_dict_wrong[(word_original, word_labels)] += 1


  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")
print(msp_type_dict_full)
print(msp_type_dict_fill)
msp_word_dict_full = dict(sorted(msp_word_dict_full.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_fill = dict(sorted(msp_word_dict_fill.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_wrong = dict(sorted(msp_word_dict_wrong.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict_full)
print(msp_word_dict_fill)
print(msp_word_dict_wrong)
print(sent_id_type)

{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 487, ('คับ', 'ครับ'): 466, ('คะ', 'ค่ะ'): 345, ('เบอ', 'เบอร์'): 315, ('มั้ย', 'ไหม'): 205, ('สมัค', 'สมัคร'): 186, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 136, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('โทรศัพ', 'โทรศัพท์'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('เคดิต', 'เครดิต'): 62, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'): 55, ('อยุ่', 'อยู่'): 53, ('บันชี', 'บัญชี'): 53, ('ไหม่', 'ใหม่'): 51, ('อ้ะ', 'อะ'): 49, ('รุ้', 'รู้'): 48, ('ก้อ', 'ก็'): 48, ('แร้ว', 'แล้ว'): 46, ('ใหน', 'ไหน

 51%|█████     | 2537/5000 [14:22<16:37,  2.47it/s]

err hunspell: ํา


 92%|█████████▏| 4609/5000 [26:39<02:33,  2.55it/s]

err hunspell: ์


100%|██████████| 5000/5000 [28:58<00:00,  2.88it/s]

AVG ACC: 0.0
AVG f1: 0.0
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 195
# EQUAL BLEU PREDICTION: 4805
# GLEU PREDICTION: 0.5869415673311889
# GLEU ORIGINAL: 0.5936627613265107
# CER PREDICTION: 0.11168197854890324
# CER ORIGINAL: 0.08266901295858524
{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 0, 'misspelled': 0, 'ws': 3, 'other': 0, 'abbreviation': 0, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 489, ('คับ', 'ครับ'): 467, ('เบอ', 'เบอร์'): 352, ('คะ', 'ค่ะ'): 345, ('มั้ย', 'ไหม'): 205, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('โทร', 'โทรศัพท์'): 150, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 137, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'): 55, ('ค', 'ค่ะ'): 55, ('อยุ่', 'อยู่'

# One sent

In [21]:
sentWrong = [['<s>', '▁พอ', 'สามารถช่วย', 'ผม', 'ใด้', 'ไม', '</s>'], ['<s>', '▁', 'บัตร', 'ร่วม', 'โรบินสัน', '▁', 'ไช่', 'ไม่ได้', 'หรอ', 'คับ', '</s>'], 
             ['<s>', '▁คือผม', 'เปลี่ยน', 'เบอ', 'มือถือ', 'ด้วย', 'คับ', '</s>'], ['<s>', '▁', 'ก้อ', 'ใม่', 'ได้', '</s>'], ['<s>', '▁', 'สอบถาม', 'หน่อย', 'น้ะ', 'ค้ะ', '</s>'],
             ['<s>', '▁', 'ยังไม่มี', 'บัญชี', 'กับ', 'กสิกร', '▁', 'ออก', 'ได้', 'ไม', 'ค่ะ', '</s>'], ['<s>', 'ที่', 'อยุ่', 'แบงค์', 'เลย', 'ใช่', 'มั้ย', 'ค่ะ', '</s>']]
sentCorr = [['<s>', '▁พอ', 'สามารถช่วย', 'ผม', 'ได้', 'ไหม', '</s>'], ['<s>', '▁', 'บัตร', 'ร่วม', 'โรบินสัน', '▁', '▁ใช้', 'ไม่ได้', 'เหรอ', 'ครับ', '</s>'], 
            ['<s>', '▁คือผม', 'เปลี่ยน', 'เบอร์', 'มือถือ', 'ด้วย', 'ครับ', '</s>'], ['<s>', '▁', '▁ก็', '▁ไม่', 'ได้', '</s>'], ['<s>', '▁', 'สอบถาม', 'หน่อย', 'นะ', 'คะ', '</s>'],
            ['<s>', '▁', 'ยังไม่มี', 'บัญชี', 'กับ', 'กสิกร', '▁', 'ออก', 'ได้', 'ไหม', 'คะ', '</s>'], ['<s>', 'ที่', 'อยู่', 'แบงค์', 'เลย', 'ใช่', 'ไหม', 'คะ', '</s>']]

textList = []
labelList = []
maskList = []
for i in range(7):
  textList.append(tokenizer.convert_tokens_to_ids(sentWrong[i]))
  labelList.append(tokenizer.convert_tokens_to_ids(sentCorr[i]))
  maskList.append([1 for k in textList[i]])

print(len(textList), len(labelList), len(maskList))

7 7 7


In [23]:
asd = ['ที่', 'อุ่ย', 'แบงค์', 'เลย', 'ใช่', 'มั้ย', 'ค่ะ']
print("|".join(asd))

ที่|อุ่ย|แบงค์|เลย|ใช่|มั้ย|ค่ะ


In [22]:
msp_type_dict = {}
msp_type_dict_full = {}
msp_type_dict_fill = {}
sent_id_type = {}
msp_word_dict = {}
msp_word_dict_full = {}
msp_word_dict_fill = {}

len_msp_type = len(msp_type)
for i in range(len_msp_type):
  len_a = len(msp_type[i])
  for j in range(len_a):
    k = msp_type[i][j][0]
    if not k in msp_type_dict:
      msp_type_dict[k] = 1
      msp_type_dict_full[k] = 0
      msp_type_dict_fill[k] = 0
      sent_id_type[k] = []
    else:
      msp_type_dict[k] += 1
    
    len_b = len(msp_word[i][j])
    for k in range(len_b):
      x = msp_word[i][j][k]
      if not x in msp_word_dict:
        msp_word_dict[x] = 1
        msp_word_dict_full[x] = 0
        msp_word_dict_fill[x] = 0
      else:
        msp_word_dict[x] += 1

msp_word_dict_wrong = msp_word_dict_fill.copy()
print(msp_type_dict)
print(msp_type_dict_fill)
print(msp_type_dict_full)
msp_word_dict = dict(sorted(msp_word_dict.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict)
print(msp_word_dict_fill)
print(msp_word_dict_full)

bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

NUM_SAMPLE=7
for i in tqdm(range(NUM_SAMPLE)):
  text_id = textList[i]
  mask = maskList[i]
  labels_id = labelList[i]

  # text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  # text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  # labels_id = ds_mlm.iloc[i]['labels']
  # labels_id = [k for k in labels_id if k != 1]
  # mask = ds_mlm.iloc[i]['attention_mask']
  # mask = [k for k in mask if k != 0]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]

  i_f = evaluate_one_text(tagging_model, text_id, mask, labels_id)
  # print(i_f)
  i_f_len = len(i_f)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(i_f_len):
    text_token[j+1] = text_token[j+1].replace("▁", " ")
    if(i_f[j] == 'i'):    
      if langid.classify(text_token[j+1])[0] == 'th':
        try:
          corrected = spell(text_token[j+1],"th_TH")
        except:
          print(f"err hunspell: {text_token[j+1]}")
        else:
          if not corrected == "No Suggestions":
            # print(f"{text_token[j+1]} => {corrected[0]}")
            text_token[j+1] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for j in range(labels_len):
    if not text_token[j] == original_token[j]: #change
      denom += 1
      if text_token[j] == labels_token[j]:
        numer += 1
        TP += 1
      elif not text_token[j] == labels_token[j]:
        FP += 1
    elif text_token[j] == original_token[j]: #no change
      if text_token[j] == labels_token[j]:
        TN += 1
      elif not text_token[j] == labels_token[j]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  print(references)
  print(original)
  print(predictions)

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred


  # print(f"MSP TYPE: {msp_type[i]}")
  original_id = text_id.copy()
  labels = labels_id.copy()
  predicted_id = tokenizer.convert_tokens_to_ids(text_token)
  # print(f"ORIG ({len(original_id)}): {original_id}")
  # print(f"PRED ({len(predicted_id)}): {predicted_id}")
  # print(f"REFE ({len(labels)}): {labels}")
  # for w in msp_type[i]:
  #   msp_type_dict_full[w[0]] += 1
  #   flag = 0
  #   a = []
  #   for j in range(w[1]+1, w[2]+1):
  #     a.append((original_id[j], predicted_id[j]))
  #     # print(tokenizer.convert_ids_to_tokens(predicted_id[j]), tokenizer.convert_ids_to_tokens(labels[j]))
  #     if predicted_id[j] != labels[j]:
  #       flag = 1
  #       break
  #   if flag == 0:
  #     msp_type_dict_fill[w[0]] += 1
  #     if i not in sent_id_type[w[0]]: #it may look too little because it only takes full marks
  #       sent_id_type[w[0]].append([i, a])


  for j in range(labels_len):
    if labels[j] != original_id[j]: #change
      word_original = tokenizer.convert_ids_to_tokens(original_id[j])
      word_original = word_original.replace("▁", "")
      word_predicted = tokenizer.convert_ids_to_tokens(predicted_id[j])
      word_predicted = word_predicted.replace("▁", "")
      word_labels = tokenizer.convert_ids_to_tokens(labels[j])
      word_labels = word_labels.replace("▁", "")
      if (word_original, word_labels) in msp_word_dict_fill: #have key
        msp_word_dict_full[(word_original, word_labels)] += 1
        if labels[j] == predicted_id[j]: #correct
          msp_word_dict_fill[(word_original, word_labels)] += 1
        elif labels[j] != predicted_id[j]: #incorrect
          msp_word_dict_wrong[(word_original, word_labels)] += 1


  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")
print(msp_type_dict_full)
print(msp_type_dict_fill)
msp_word_dict_full = dict(sorted(msp_word_dict_full.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_fill = dict(sorted(msp_word_dict_fill.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_wrong = dict(sorted(msp_word_dict_wrong.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict_full)
print(msp_word_dict_fill)
print(msp_word_dict_wrong)
print(sent_id_type)

{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 487, ('คับ', 'ครับ'): 466, ('คะ', 'ค่ะ'): 345, ('เบอ', 'เบอร์'): 315, ('มั้ย', 'ไหม'): 205, ('สมัค', 'สมัคร'): 186, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 136, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('โทรศัพ', 'โทรศัพท์'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('เคดิต', 'เครดิต'): 62, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'): 55, ('อยุ่', 'อยู่'): 53, ('บันชี', 'บัญชี'): 53, ('ไหม่', 'ใหม่'): 51, ('อ้ะ', 'อะ'): 49, ('รุ้', 'รู้'): 48, ('ก้อ', 'ก็'): 48, ('แร้ว', 'แล้ว'): 46, ('ใหน', 'ไหน

 14%|█▍        | 1/7 [00:00<00:04,  1.39it/s]

[[['<s>', ' พอ', 'สามารถช่วย', 'ผม', 'ได้', 'ไหม', '</s>']]]
[['<s>', ' พอ', 'สามารถช่วย', 'ผม', 'ใด้', 'ไม', '</s>']]
[['<s>', ' พอ', 'สามารถช่วย', 'ผม', 'ใด', 'ไม้', '</s>']]


 29%|██▊       | 2/7 [00:01<00:02,  1.79it/s]

[[['<s>', ' ', 'บัตร', 'ร่วม', 'โรบินสัน', ' ', ' ใช้', 'ไม่ได้', 'เหรอ', 'ครับ', '</s>']]]
[['<s>', ' ', 'บัตร', 'ร่วม', 'โรบินสัน', ' ', 'ไช่', 'ไม่ได้', 'หรอ', 'คับ', '</s>']]
[['<s>', ' ', 'บัตร', 'ร่วม', 'โรบินสัน', ' ', 'ไช', 'ไม่ได้', 'หรอ', 'คับ', '</s>']]


...

[[['<s>', ' ', 'สอบถาม', 'หน่อย', 'นะ', 'คะ', '</s>']]]
[['<s>', ' ', 'สอบถาม', 'หน่อย', 'น้ะ', 'ค้ะ', '</s>']]
[['<s>', ' ', 'สอบถาม', 'หน่อย', 'นะ', 'คะ', '</s>']]


 86%|████████▌ | 6/7 [00:02<00:00,  3.12it/s]

[[['<s>', ' ', 'ยังไม่มี', 'บัญชี', 'กับ', 'กสิกร', ' ', 'ออก', 'ได้', 'ไหม', 'คะ', '</s>']]]
[['<s>', ' ', 'ยังไม่มี', 'บัญชี', 'กับ', 'กสิกร', ' ', 'ออก', 'ได้', 'ไม', 'ค่ะ', '</s>']]
[['<s>', ' ', 'ยังไม่มี', 'บัญชี', 'กับ', 'กสิกร', ' ', 'ออก', 'ได้', 'ไม้', 'ค่ะ', '</s>']]


100%|██████████| 7/7 [00:02<00:00,  2.63it/s]

[[['<s>', 'ที่', 'อยู่', 'แบงค์', 'เลย', 'ใช่', 'ไหม', 'คะ', '</s>']]]
[['<s>', 'ที่', 'อยุ่', 'แบงค์', 'เลย', 'ใช่', 'มั้ย', 'ค่ะ', '</s>']]
[['<s>', 'ที่', 'อุ่ย', 'แบงค์', 'เลย', 'ใช่', 'มั้ย', 'ค่ะ', '</s>']]
AVG ACC: 0.09523809523809523
AVG f1: 0.1142857142857143
# HIGHER BLEU PREDICTION: 1
# LOWER BLEU PREDICTION: 0
# EQUAL BLEU PREDICTION: 6
# GLEU PREDICTION: 0.5450990112644248
# GLEU ORIGINAL: 0.4736704398358534
# CER PREDICTION: 0.14492121634978777
# CER ORIGINAL: 0.11914028755133489
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{('ค่ะ', 'คะ'): 2, ('คับ', 'ครับ'): 2, ('ไม', 'ไหม'): 2, ('อยุ่', 'อยู่'): 1, ('เบอ', 'เบอร์'): 1, ('มั้ย', 'ไหม'): 1, ('หรอ', 'เหรอ'): 1, ('ใด้', 'ได้'): 1, ('ก้อ', 'ก็'): 1, ('ค้ะ', 'คะ'): 1, ('ใม่', 'ไม่'): 1, ('น้ะ', 'นะ'): 1, ('ไช่', 'ใช้'): 1, ('มั๊ย', 'ไหม'): 0, ('ก้', 'ก็'): 0, ('ตุ้', 'ตู้'): 0, ('แอฟ', 'แอพ'): 0, ('หรอ', 'หรื

In [ ]:
for key in sent_id_type:
  print(f"=========[{key}]==========")
  for sent_id, words in sent_id_type[key]:
    text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
    labels = ds_mlm.iloc[sent_id]['labels']
    text = [k for k in text if k != 1]
    labels = [k for k in labels if k != 1]
    original = [ids_to_tokens(text)]
    references = [[ids_to_tokens(labels)]]
    for i in words:
      print(f"MSP: {tokenizer.convert_ids_to_tokens(i[0])} => {tokenizer.convert_ids_to_tokens(i[1])}")
    print(original)
    print(references)
    print("-------------------")

=========[morphed]==========
=========[misspelled]==========
=========[ws]==========
MSP: เป็นไร => เป็นไร
[['<s>', '▁', 'อ่อ', 'ไม่', 'เป็นไร', 'แล้ว', 'คับ', '</s>']]
[[['<s>', '▁', '▁อ๋อ', 'ไม่', 'เป็นไร', 'แล้ว', 'ครับ', '</s>']]]
-------------------
MSP: ตรงกัน => ตรงกัน
[['<s>', '▁', 'เบอ', '▁', 'ตรงกัน', 'ครับ', '</s>']]
[[['<s>', '▁', 'เบอร์', '▁', 'ตรงกัน', 'ครับ', '</s>']]]
-------------------
MSP: ก้อง => ก้อง
[['<s>', '▁เออ', 'เรา', 'ก้อง', 'ง', '▁', 'สิ', '้', 'ค้ะ', '</s>']]
[[['<s>', '▁เออ', 'เรา', 'ก้อง', 'ง', '▁', 'สิ', '_', 'คะ', '</s>']]]
-------------------
=========[other]==========
=========[abbreviation]==========
=========[new]==========


In [ ]:
pickle_hunspell_tagging_json = {}
pickle_hunspell_tagging_json['msp_type_dict_full'] = msp_type_dict_full
pickle_hunspell_tagging_json['msp_type_dict_fill'] = msp_type_dict_fill
pickle_hunspell_tagging_json['msp_word_dict_full'] = msp_word_dict_full
pickle_hunspell_tagging_json['msp_word_dict_fill'] = msp_word_dict_fill
pickle_hunspell_tagging_json['msp_word_dict_wrong'] = msp_word_dict_wrong
pickle_hunspell_tagging_json['sent_id_type'] = sent_id_type

pickle.dump(pickle_hunspell_tagging_json, open('pickle_hunspell_tagging_json.pkl', 'wb'))

In [ ]:
pickle_hunspell_tagging_json = pickle.load(open('drive/MyDrive/AIBuilders/json/pickle_hunspell_tagging_json.pkl', 'rb'))
msp_type_dict_full = pickle_hunspell_tagging_json['msp_type_dict_full']
msp_type_dict_fill = pickle_hunspell_tagging_json['msp_type_dict_fill']
msp_word_dict_full = pickle_hunspell_tagging_json['msp_word_dict_full']
msp_word_dict_fill = pickle_hunspell_tagging_json['msp_word_dict_fill']
msp_word_dict_wrong = pickle_hunspell_tagging_json['msp_word_dict_wrong']
sent_id_type = pickle_hunspell_tagging_json['sent_id_type']

In [ ]:
for key in sent_id_type:
  print(f"=========[{key}]==========")
  for sent_id, words in sent_id_type[key]:
    text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
    labels = ds_mlm.iloc[sent_id]['labels']
    text = [k for k in text if k != 1]
    labels = [k for k in labels if k != 1]
    original = [ids_to_tokens(text)]
    references = [[ids_to_tokens(labels)]]
    for i in words:
      print(f"MSP: {tokenizer.convert_ids_to_tokens(i[0])} => {tokenizer.convert_ids_to_tokens(i[1])}")
    print(original)
    print(references)
    print("-------------------")

=========[morphed]==========
=========[misspelled]==========
=========[ws]==========
MSP: เป็นไร => เป็นไร
[['<s>', '▁', 'อ่อ', 'ไม่', 'เป็นไร', 'แล้ว', 'คับ', '</s>']]
[[['<s>', '▁', '▁อ๋อ', 'ไม่', 'เป็นไร', 'แล้ว', 'ครับ', '</s>']]]
-------------------
MSP: ตรงกัน => ตรงกัน
[['<s>', '▁', 'เบอ', '▁', 'ตรงกัน', 'ครับ', '</s>']]
[[['<s>', '▁', 'เบอร์', '▁', 'ตรงกัน', 'ครับ', '</s>']]]
-------------------
MSP: ก้อง => ก้อง
[['<s>', '▁เออ', 'เรา', 'ก้อง', 'ง', '▁', 'สิ', '้', 'ค้ะ', '</s>']]
[[['<s>', '▁เออ', 'เรา', 'ก้อง', 'ง', '▁', 'สิ', '_', 'คะ', '</s>']]]
-------------------
=========[other]==========
=========[abbreviation]==========
=========[new]==========
